### Load workspace

In [16]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

Found the config file in: /config.json


### Load datastore and get dataset path

In [3]:
from azure.ai.ml.entities import Data

dataset_name = "heart-data"
dataset_version = "1"

dataset = ml_client.data.get(name=dataset_name, version=dataset_version)
dataset_path = dataset.path

### Load dataset to dataframe

In [4]:
import pandas as pd

df = pd.read_csv(dataset_path)

### Prepare data

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

### Train model

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy:", accuracy)

Model accuracy: 0.8064066852367688


### Save model

In [8]:
import joblib
import os

model_dir = "heart-disease-prediction-model-with-scaler"

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = "heart-disease-prediction-model.joblib"
scaler_path = "heart-disease-prediction-scaler.joblib"

joblib.dump(model, f"{model_dir}/{model_path}")
joblib.dump(scaler, f"{model_dir}/{scaler_path}")

['heart-disease-prediction-model-with-scaler/heart-disease-prediction-scaler.joblib']

### Test saved model

In [12]:
import joblib
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

model = joblib.load(f"{model_dir}/{model_path}")
scaler = joblib.load(f"{model_dir}/{scaler_path}")

data = np.array([[58,0,0,100,248,0,0,122,0,1,1,0,2]])
data_scaled = scaler.transform(data)

prediction = model.predict_proba(data_scaled)
print(round(prediction[0][1] * 100, 2))

77.35


### Register model

In [13]:
from azure.ai.ml.entities import Model

model = Model(
    name="heart-disease-prediction-model-with-scaler",
    path="heart-disease-prediction-model-with-scaler",
    type="custom_model",
    description="Heart disease model with scaler"
)

registered_model = ml_client.models.create_or_update(model)

Uploading heart-disease-prediction-model-with-scaler (0.0 MBs): 100%|██████████| 2318/2318 [00:00<00:00, 93195.14it/s]




### Create and prepare environment

In [14]:
from azure.ai.ml.entities import Environment

environment = Environment(
    name="heart-disease-prediction-env",
    description="Environment for heart disease prediction",
    conda_file={
        "name": "heart-disease-prediction-env",
        "channels": ["defaults"],
        "dependencies": [
            "python=3.8",
            "pip",
            {
                "pip": [
                    "azureml-defaults",
                    "joblib",
                    "numpy",
                    "scikit-learn"
                ]
            }
        ]
    },
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04:latest"
)

created_environment = ml_client.environments.create_or_update(environment)

### Create endpoint

In [18]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint_name = "heart-disease-endpoint"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint for heart disease prediction",
    traffic={"default": 100}
)

operation = ml_client.online_endpoints.begin_create_or_update(endpoint)
endpoint_result = operation.result()

HttpResponseError: (SubscriptionNotRegistered) Resource provider [N/A] isn't registered with Subscription [N/A]. Please see troubleshooting guide, available here: https://aka.ms/register-resource-provider
Code: SubscriptionNotRegistered
Message: Resource provider [N/A] isn't registered with Subscription [N/A]. Please see troubleshooting guide, available here: https://aka.ms/register-resource-provider

### Deploy endpoint

In [105]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

code_config = CodeConfiguration(
    code="./",
    scoring_script="score.py"
)

deployment = ManagedOnlineDeployment(
    name="default",
    endpoint_name=endpoint_name,
    model=registered_model,
    environment=created_environment.id,
    code_configuration=code_config,
    instance_type="Standard_F2s_v2",
    instance_count=1
)

operation = ml_client.online_deployments.begin_create_or_update(deployment)
deployment_result = operation.result()

Check: endpoint heart-disease-endpoint exists
Uploading kubabednarekk (0.03 MBs): 100%|██████████| 32694/32694 [00:00<00:00, 709176.91it/s]




.........................................................................

### Route traffic to deployment

In [106]:
from azure.ai.ml.entities import OnlineEndpoint

endpoint = ml_client.online_endpoints.get(name=endpoint_name)

endpoint.traffic = {"default": 100}
updated_endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


### Test model

In [111]:
import requests
import json

endpoint_keys = keys = ml_client.online_endpoints.get_keys(name=endpoint_name)

test_data = json.dumps({"data": [[61,0,0,145,307,0,0,146,1,1,1,0,3]]})
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {endpoint_keys.primary_key}"}

response = requests.post(endpoint_result.scoring_uri, data=test_data, headers=headers)

print("URI punktu końcowego: ",  endpoint_result.scoring_uri)
print("Klucz API punktu końcowego: ",  endpoint_keys.primary_key)
print("Odpowiedź punktu końcowego:", response.json())

Odpowiedź punktu końcowego: {'probability': 16.05}
